In [1]:
import torch
import pandas as pd
import os
from torchvision import transforms
from EmotionDataset import EmotionDataset, get_transform
from EmotionCNN import EmotionCNN
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam
from sklearn.model_selection import train_test_split

In [2]:
# Load and prepare the dataset
dataset_path = os.path.join(os.getcwd(), 'data')
labels_path = os.path.join(dataset_path, 'labels.csv')
df = pd.read_csv(labels_path)

df['label'] = df['pth'].apply(lambda x: x.split('/')[0])
df['pth'] = df['pth'].apply(lambda x: os.path.join(dataset_path, x))
one_hot_encoded_df = pd.get_dummies(df['label']).astype(int)
df = pd.concat([df.drop('label', axis=1), one_hot_encoded_df], axis=1)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
df.drop(columns=['Unnamed: 0'], inplace=True)
df.drop(columns=['relFCs'], inplace=True)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [3]:
# Create the datasets
transform = get_transform()
train_dataset = EmotionDataset(train_df, transform)

In [4]:
# Set up the model and training parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EmotionCNN().to(device)
criterion = nn.BCEWithLogitsLoss()  # Changed loss function for one-hot encoded labels
optimizer = Adam(model.parameters(), lr=0.001)

In [5]:
# DataLoader with custom collate function for handling tensor operations
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [8]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/10], Loss: 0.2066
Epoch [2/10], Loss: 0.2438
Epoch [3/10], Loss: 0.1555
Epoch [4/10], Loss: 0.1717
Epoch [5/10], Loss: 0.1626
Epoch [6/10], Loss: 0.0205
Epoch [7/10], Loss: 0.0526
Epoch [8/10], Loss: 0.0362
Epoch [9/10], Loss: 0.0662
Epoch [10/10], Loss: 0.0401


In [11]:
torch.save(model, './models/emotionModel1')